In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key
%matplotlib inline

In [2]:
%pip install gmaps

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Punta Arenas,CL,2022-06-13 18:13:25,-53.1500,-70.9167,48.31,71,0,13.80
1,1,Jvari,GE,2022-06-13 18:13:25,42.7169,42.0520,67.32,98,97,2.51
2,2,Cape Town,ZA,2022-06-13 18:13:25,-33.9258,18.4232,55.11,94,75,5.75
3,3,Wanning,CN,2022-06-13 18:13:25,18.8003,110.3967,81.01,93,100,10.47
4,4,Hasaki,JP,2022-06-13 18:13:26,35.7333,140.8333,61.29,72,100,14.03


In [4]:
#Get the datatype
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [5]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [9]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [11]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [14]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
3,3,Wanning,CN,2022-06-13 18:13:25,18.8003,110.3967,81.01,93,100,10.47
5,5,Butaritari,KI,2022-06-13 18:13:26,3.0707,172.7902,81.52,73,30,11.79
6,6,Jamestown,US,2022-06-13 18:13:26,42.0970,-79.2353,80.53,30,34,5.99
12,12,Hilo,US,2022-06-13 18:13:28,19.7297,-155.0900,84.27,75,0,4.61
13,13,Orange Cove,US,2022-06-13 18:13:28,36.6244,-119.3137,78.62,42,8,7.65
18,18,Hithadhoo,MV,2022-06-13 18:13:30,-0.6000,73.0833,83.30,73,46,6.15
26,26,Atascadero,US,2022-06-13 18:13:32,35.4894,-120.6707,79.36,51,0,4.00
29,29,Samarai,PG,2022-06-13 18:13:34,-10.6167,150.6667,81.59,75,75,12.86
34,34,Kapaa,US,2022-06-13 18:13:35,22.0752,-159.3190,79.12,81,0,18.41
39,39,Atuona,PF,2022-06-13 18:13:36,-9.8000,-139.0333,78.08,66,62,19.15


In [15]:
preferred_cities_df.count()

City_ID       169
City          169
Country       169
Date          169
Lat           169
Lng           169
Max Temp      169
Humidity      169
Cloudiness    169
Wind Speed    169
dtype: int64

In [21]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Wanning,CN,81.01,18.8003,110.3967,
5,Butaritari,KI,81.52,3.0707,172.7902,
6,Jamestown,US,80.53,42.0970,-79.2353,
12,Hilo,US,84.27,19.7297,-155.0900,
13,Orange Cove,US,78.62,36.6244,-119.3137,
18,Hithadhoo,MV,83.30,-0.6000,73.0833,
26,Atascadero,US,79.36,35.4894,-120.6707,
29,Samarai,PG,81.59,-10.6167,150.6667,
34,Kapaa,US,79.12,22.0752,-159.3190,
39,Atuona,PF,78.08,-9.8000,-139.0333,


In [22]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [33]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [34]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Wanning,CN,81.01,18.8003,110.3967,Paradise Hotel
5,Butaritari,KI,81.52,3.0707,172.7902,Isles Sunset Lodge
6,Jamestown,US,80.53,42.0970,-79.2353,Hampton Inn & Suites Jamestown
12,Hilo,US,84.27,19.7297,-155.0900,Hilo Hawaiian Hotel
13,Orange Cove,US,78.62,36.6244,-119.3137,
18,Hithadhoo,MV,83.30,-0.6000,73.0833,Scoop Guest House
26,Atascadero,US,79.36,35.4894,-120.6707,"Holiday Inn Express & Suites Atascadero, an IH..."
29,Samarai,PG,81.59,-10.6167,150.6667,Nuli Sapi
34,Kapaa,US,79.12,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
39,Atuona,PF,78.08,-9.8000,-139.0333,Villa Enata


In [36]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [38]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [39]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [40]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))